In [10]:

"""
Briar Doty
AMATH 584 - midterm
11/6/20
Full repository available at https://github.com/briardoty/amath584/midterm
"""

%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import math
from functools import reduce

In [ ]:
# V - LU decomposition without pivoting
def lu_decomp(A):
    
    # only works for square matrices
    m, n = A.shape
    if m != n:
        return -1
    
    # define result matrices
    L, U = np.zeros_like(A), np.copy(A)
    L_arr = []
    
    for k in range(m):
        
        # start with I for L_k
        L_k = np.identity(m)
        
        # update kth column below diagonals
        for j in range(k+1,m):
            L_k[j,k] = -U[j,k]/U[k,k]
            
        # store L_k
        L_arr.append(L_k)
        
        # update "A" using L_k, building U in the process
        U = L_k@U
         
    # update L
    L = np.copy(L_arr[0])
    L = np.linalg.inv(L)
    for Li in L_arr[1:]:
        Linv = np.linalg.inv(Li)
        L = L@Linv
        
    return L, U

In [115]:
# Va - demonstrate functionality on pre-defined matrix
A = np.array([
    [1.,-2.,0],
    [-3.,-2.,1],
    [6.,0.,3.]
])

L, U = lu_decomp(A)
A_lu = L@U
print(f"L:\n {L}\n")
print(f"U:\n {U}\n")
print(f"A_lu:\n {A_lu}\n")

print(f"Norm(A - A_lu): {np.linalg.norm(A - A_lu)}")

Pivot index: 2
Pivot index: 2
L:
 [[ 0.16666667  1.          0.        ]
 [-0.5         1.          1.        ]
 [ 1.          0.          0.        ]]

U:
 [[ 6.   0.   3. ]
 [ 0.  -2.  -0.5]
 [ 0.   0.   3. ]]

A_lu:
 [[ 1. -2.  0.]
 [-3. -2.  1.]
 [ 6.  0.  3.]]

Norm(A - A_lu): 0.0


In [98]:
# Vc - demonstrate functionality on random matrix
m = 4
A = np.random.rand(m,m)

L, U = lu_decomp(A)
A_lu = L@U
print(f"L:\n {L}\n")
print(f"U:\n {U}\n")
print(f"A_lu:\n {A_lu}\n")

print(f"Norm(A - A_lu): {np.linalg.norm(A - A_lu)}")

Pivot index: 1
Pivot index: 3
Pivot index: 3
L:
 [[0.76553718 1.3798205  1.         0.        ]
 [1.         0.         0.         0.        ]
 [0.96375215 0.62805159 0.04221871 1.        ]
 [0.53523064 1.         0.         0.        ]]

U:
 [[ 7.22900626e-01  4.19747048e-02  2.30791096e-01  3.70209618e-01]
 [ 5.55111512e-17  6.42352586e-01  7.41574419e-01  3.55857785e-01]
 [-7.65954245e-17  0.00000000e+00 -1.06952465e+00 -5.54752001e-01]
 [-3.16301066e-17  0.00000000e+00  0.00000000e+00  9.35117412e-02]]

A_lu:
 [[0.5534073  0.91846446 0.1303941  0.21967709]
 [0.72290063 0.0419747  0.2307911  0.37020962]
 [0.69669703 0.44388378 0.64301845 0.65037819]
 [0.38691856 0.66481873 0.86510088 0.55400531]]

Norm(A - A_lu): 0.236975412014138


In [111]:
# V - LU decomposition with pivoting
def lu_decomp(A):
    
    # only works for square matrices
    m, n = A.shape
    if m != n:
        return -1
    
    # define result matrices
    L, U = np.zeros_like(A), np.copy(A)
    L_arr = []
    P_arr = []
    
    # loop over columns
    for k in range(m):
        
        # start with I for L_k
        L_k = np.identity(m)
        
        # choose pivot
        P_k = np.identity(m)
        if k < m - 1:
            pi = k + 1 + np.argmax(np.abs(U[k+1:,k]))
#             print(f"Pivot index: {pi}")

            # define permutation
            row_k = np.array(P_k[k])
            P_k[k] = P_k[pi]
            P_k[pi] = row_k
            #print(f"Perm:\n {P_k}\n")

            # apply permutation
            U = P_k@U
            
        # store P_k
        P_arr.append(P_k)
        
        # update each row below diagonal in kth col
        for j in range(k+1,m):
            L_k[j,k] = -U[j,k]/U[k,k]
            
        # L_k
        L_arr.append(L_k)
        
        # update "A" using L_k, building U in the process
        U = L_k@U
         
    # find L
#     L_arr.reverse()
#     P_arr.reverse()
    
    for i in range(1, len(P_arr)):
        L_arr[i-1] = P_arr[i]@L_arr[i-1]@np.linalg.inv(P_arr[i])
        
    P_inv = P_arr[0]
    for pi in P_arr[1:-1]:
        P_inv = pi@P_inv
    
    L_pri = L_arr[0]
    for li in L_arr[1:-1]:
        L_pri = li@L_pri
        
    L_inv = L_pri@P_inv
    

#     for i in range(1, len(L_arr)):
#         L_arr[i] = P_arr[i-1]@L_arr[i]@np.linalg.inv(P_arr[i-1])
    
#     Linv = np.copy(L_arr[0])
#     for Li in L_arr[1:]:
#         Linv = Linv@Li
        
#     for Pi in P_arr:
#         Linv = Linv@Pi
        
    L = np.linalg.inv(L_inv)
    
#     L = np.copy(L_arr[0])
#     L = np.linalg.inv(L)
#     for Li in L_arr[1:]:
#         Linv = np.linalg.inv(Li)
#         L = L@Linv
        
    return L, U

In [116]:
# Vb - demonstrate functionality on pre-defined matrix with 0 on diag
A = np.array([
    [0.,-2.,0,7],
    [-3.,1.,1,1],
    [6.,2.,3.,2],
    [2.,1.,-3.,-1]
])

L, U = lu_decomp(A)
A_lu = L@U
print(f"L:\n {L}\n")
print(f"U:\n {U}\n")
print(f"A_lu:\n {A_lu}\n")

print(f"Norm(A - A_lu): {np.linalg.norm(A - A_lu)}")

Pivot index: 2
Pivot index: 2
Pivot index: 3
L:
 [[-5.02742502e-17  1.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 3.33333333e-01 -1.00000000e+00 -6.25000000e-01  1.00000000e+00]
 [ 1.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-5.00000000e-01 -1.66666667e-01  1.00000000e+00  7.50622485e-18]]

U:
 [[ 6.      2.      3.      2.    ]
 [ 0.     -2.      0.      7.    ]
 [ 0.      0.     -4.     -0.5   ]
 [ 0.      0.      0.      8.6875]]

A_lu:
 [[-3.01645501e-16 -2.00000000e+00 -1.50822751e-16  7.00000000e+00]
 [ 2.00000000e+00  2.66666667e+00  3.50000000e+00  2.66666667e+00]
 [ 6.00000000e+00  2.00000000e+00  3.00000000e+00  2.00000000e+00]
 [-3.00000000e+00 -6.66666667e-01 -5.50000000e+00 -2.66666667e+00]]

Norm(A - A_lu): 8.579691784155834
